In [3]:
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials
import pandas as pd
import numpy as np

In [4]:
# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1TgGhJQO9XfWN9swtD-L8emANCFP3-MX__Gc6iLnPbW8'
SAMPLE_RANGE_NAME = 'Merge Centros, Horario e Info Vacunacion'

In [5]:
def process_data(df):
    df = df[df['DISPONIBILIDAD']=='SI']
    return df

In [6]:
creds = Credentials.from_service_account_file('credentials.json')

In [7]:

def get_sheets_data():
    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
    values = result.get('values', [])

    
    if not values:
        print('No data found.')
    else:
        df = pd.DataFrame(values[1:], columns=values[0])
    return df

In [8]:
df = get_sheets_data()

In [9]:
df.shape

(368, 23)

In [10]:
df.head(1).T

,0
CENTRO,Asamblea de Colegios Profesionales
VACUNA,MODERNA
DOSIS,PRIMERA
GRUPO,Mayores de 25 años
REQUISITO,DPI + REGISTRO
DISPONIBILIDAD,SI
DEPARTAMENTOS,GUATEMALA
MUNICIPIO,GUATEMALA
DIRECCIÓN,"0 calle 15-46, zona 15 Colonia El Maestro"
AFLUENCIA,POCA


In [11]:
def arrange_grouped_vaccines(vaccines):
    vaccine_name = vaccines['VACUNA'].iloc[0]
    vaccine_dose = vaccines['DOSIS'].iloc[0]
    if len(vaccine_name) < 2:
        return ''
    vaccine_dict = {}
    groups = vaccines[['GRUPO',
                     'REQUISITO',
                     'DISPONIBILIDAD']].to_dict('records')
    vaccine_dict[f'{vaccine_name} {vaccine_dose}'] = groups
    
    
#     print(type(vaccine_dict))
    

    return vaccine_dict

In [12]:
def arrange_grouped_centers(center):
    center_days = center[['LUNES','MARTES', 'MIERCOLES', 'JUEVES', 'VIERNES', 'SABADO','DOMINGO']].iloc[0]
    center_ex = center.drop(['LUNES',
                             'MARTES',
                             'MIERCOLES',
                             'JUEVES',
                             'VIERNES',
                             'SABADO',
                             'DOMINGO',
                             'VACUNA',
                             'DOSIS',
                             'GRUPO',
                             'REQUISITO',
                             'DISPONIBILIDAD',
#                              'CENTRO'
                            ], axis=1).iloc[0]
    center_dict = center_ex.to_dict()
#     center_dict[center['CENTRO'].iloc[0]] = center_ex.to_dict()
    
    center_dict['DIAS'] = {
        'LUNES' :     np.where(center_days['LUNES'] == 'SI',True,False).item(),
        'MARTES' :    np.where(center_days['MARTES'] == 'SI',True,False).item(),
        'MIERCOLES' : np.where(center_days['MIERCOLES'] == 'SI',True,False).item(),
        'JUEVES' :    np.where(center_days['JUEVES'] == 'SI',True,False).item(),
        'VIERNES' :   np.where(center_days['VIERNES'] == 'SI',True,False).item(),
        'SABADO' :    np.where(center_days['SABADO'] == 'SI',True,False).item(),
        'DOMINGO' :   np.where(center_days['DOMINGO'] == 'SI',True,False).item(),
    }
    center_dict['HORARIO ENTRE SEMANA'] = center_ex['HORARIO ENTRE SEMANA']
    center_dict['HORARIO ENTRE SEMANA'] = center_ex['HORARIO ENTRE SEMANA']
    vacunas = center.groupby('VACUNA').apply(arrange_grouped_vaccines).to_dict()
    if vacunas.get('', False) == False:
        center_dict['VACUNAS'] = vacunas
#     pprint.pprint(center_dict)
    return center_dict

In [13]:
df[df['CENTRO']=='Centro Comercial Pradera']['VACUNA']

45    
Name: VACUNA, dtype: object

In [14]:
formatted_data = df.groupby('CENTRO').apply(arrange_grouped_centers)

In [15]:
formatted_data.values[0]

{'CENTRO': '5ta Brigada de Infantería, Huehuetenango',
 'DEPARTAMENTOS': 'HUEHUETENANGO',
 'MUNICIPIO': 'HUEHUETENANGO',
 'DIRECCIÓN': 'Las Lagunas zona 10, Huehuetenango',
 'AFLUENCIA': 'MUCHA',
 'INGRESO': 'A PIE',
 'OBSERVACIONES': 'Lunes a Viernes —> 6pm - 12am / \n\nFB: https://www.facebook.com/Quinta-Brigada-De-Infanter%C3%ADa-MGS-1945268509018288/ ',
 'Google Maps': 'https://www.google.com/maps/place/Quinta+Brigada+de+Infanteria+%22Mariscal+Gregorio+Solares%22/@15.3095029,-91.5241311,17z/data=!3m1!4b1!4m5!3m4!1s0x858c3e0059f8fe0b:0xcd5f8c372cb0a0e6!8m2!3d15.3095029!4d-91.521937',
 'DISPONIBILIDAD CENTRO': 'SI',
 'HORARIO ENTRE SEMANA': '6pm - 12am',
 'HORARIO FIN DE SEMANA': '',
 'DIAS': {'LUNES': True,
  'MARTES': True,
  'MIERCOLES': True,
  'JUEVES': True,
  'VIERNES': True,
  'SABADO': False,
  'DOMINGO': False},
 'VACUNAS': {'SPUTNIK': {'SPUTNIK PRIMERA': [{'GRUPO': 'Registrados',
     'REQUISITO': 'DPI + REGISTRO',
     'DISPONIBILIDAD': 'SI'}]}}}

In [16]:

formatted_data['Centro Comercial Pradera']

{'CENTRO': 'Centro Comercial Pradera',
 'DEPARTAMENTOS': 'GUATEMALA',
 'MUNICIPIO': 'GUATEMALA',
 'DIRECCIÓN': 'Blvd. los Proceres 25-85, zona 10',
 'AFLUENCIA': 'NO SE SABE',
 'INGRESO': 'A PIE',
 'OBSERVACIONES': 'PENDIENTE DE MAYOR INFORMACIÓN - aún no está habilitado ',
 'Google Maps': 'https://www.google.com/maps/place/Galerias+La+Pradera/@14.5796604,-90.4973609,17z/data=!3m1!4b1!4m5!3m4!1s0x8589a39bada6ad3d:0x2b90f785d3d041a6!8m2!3d14.5796604!4d-90.4951722',
 'DISPONIBILIDAD CENTRO': 'SI',
 'HORARIO ENTRE SEMANA': None,
 'HORARIO FIN DE SEMANA': None,
 'DIAS': {'LUNES': False,
  'MARTES': False,
  'MIERCOLES': False,
  'JUEVES': False,
  'VIERNES': False,
  'SABADO': False,
  'DOMINGO': False}}

In [17]:
formatted_data['Centro de Recuperación Nutricional / Anexo Centro de Salud']

{'CENTRO': 'Centro de Recuperación Nutricional / Anexo Centro de Salud',
 'DEPARTAMENTOS': 'TOTONICAPÁN',
 'MUNICIPIO': 'TOTONICAPÁN',
 'DIRECCIÓN': 'Final 17 avenida, zona 2, Totonicapán',
 'AFLUENCIA': 'NO SE SABE',
 'INGRESO': 'NO SE SABE',
 'OBSERVACIONES': 'FB Muni: https://www.facebook.com/MunicipalidadToto ',
 'Google Maps': 'https://www.google.com/maps/place/Centro+de+Salud+de+Totonicap%C3%A1n/@14.9130983,-91.3670877,18.77z/data=!4m9!1m2!2m1!1sanexo+centro+de+salud+totonicapan!3m5!1s0x858ea5f56e6c18cb:0xc318030b2015c65c!8m2!3d14.9135377!4d-91.3668571!15sCiFhbmV4byBjZW50cm8gZGUgc2FsdWQgdG90b25pY2FwYW6SAQpnb3Zlcm5tZW50',
 'DISPONIBILIDAD CENTRO': 'SI',
 'HORARIO ENTRE SEMANA': 'Sin Confirmar',
 'HORARIO FIN DE SEMANA': None,
 'DIAS': {'LUNES': False,
  'MARTES': False,
  'MIERCOLES': False,
  'JUEVES': False,
  'VIERNES': False,
  'SABADO': False,
  'DOMINGO': False},
 'VACUNAS': {'ASTRAZENECA': {'ASTRAZENECA SEGUNDA': [{'GRUPO': 'Vacunados hace 57 días',
     'REQUISITO': 'CARNE

In [18]:
import firebase_admin
from firebase_admin import firestore
from firebase_admin import credentials
import pandas as pd
import json

cred_prod = credentials.Certificate('covid-19-bot-9b888fc679a7.json')
app_prod = firebase_admin.initialize_app(cred_prod,name='prod')

In [29]:

# cred_dev = credentials.Certificate('alma--dev-c17a0a927700.json')
# app_dev = firebase_admin.initialize_app(cred_dev,name='dev')

db = firestore.client(app=app_prod)

collection = 'centros_vacunacion'

for center in formatted_data.values:
    center_name = center['CENTRO']
    
    try:
        db.collection(collection)\
            .document(center_name.replace('/', '-'))\
            .set(center)
        print(f'Done: {center_name}')
    except:
        from pprint import pprint
        pprint(f'Error: {center_name} {center}')
  


Done: 5ta Brigada de Infantería, Huehuetenango
Done: Asamblea de Colegios Profesionales
Done: Auditorio Municipal Villa Canales
Done: CAIMI El Estor
Done: CAP San Juan Comalapa
Done: CAP de Nahualá
Done: CAP de Santa Catarina Ixtahuacán
Done: CAP de Santa María Visitación
Done: CUNOC
Done: Campo El Calvario San Juan Alotenango
Done: Campo Marte
Done: Cancha de baloncesto municipal
Done: Casa de la Cultura Zaragoza
Done: Cenma Centra Sur
Done: Centra Norte
Done: Centro Comercial El Frutal
Done: Centro Comercial La Trinidad 
Done: Centro Comercial Miraflores
Done: Centro Comercial Peri-Roosevelt
Done: Centro Comercial Portales
Done: Centro Comercial Pradera
Done: Centro Comercial Pradera Concepción
Done: Centro Comercial Pradera Huehuetenango
Done: Centro Comercial Pradera Vistares
Done: Centro Comercial Unicentro
Done: Centro Especial Alida España
Done: Centro Universitario Metropolitano -CUM-
Done: Centro Universitario de Izabal II
Done: Centro de Atención Integral (CAI), La Terminal
D